In [1]:
import os
import json
import matplotlib.pyplot as plt 
import matplotlib.image as image 
import numpy as np
import pandas as pd
import albumentations as A
import albumentations.pytorch
import cv2
import math

import torch
from pytorch_lightning import LightningModule, Trainer, LightningDataModule
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import MNIST
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

import sys
sys.path.append('../')
from utils.dataset import *
# from train import PapsClsModel
import custom_models


In [2]:
train_df = pd.read_csv( '../lbp_data/train.csv')

In [3]:
train_df.shape

(16569, 13)

In [4]:
train_df.head()

,ID,file_name,task,bbox,xmin,ymin,w,h,label,label_id,occluded,des,cell_type
0,0,patch_images/2021.01.14/LBC424-20210111(1)/LBC...,[AS6] LBC424,"[1539, 199, 139, 211]",1539,199,139,211,C,Candida,0,NaN,ASC-US
1,1,patch_images/2021.01.14/LBC424-20210111(1)/LBC...,[AS6] LBC424,"[1337, 102, 256, 136]",1337,102,256,136,AS,ASC-US,0,NaN,ASC-US
2,2,patch_images/2021.01.14/LBC424-20210111(1)/LBC...,[AS6] LBC424,"[220, 619, 166, 169]",220,619,166,169,AS,ASC-US,0,NaN,ASC-US
3,3,patch_images/2021.01.14/LBC424-20210111(1)/LBC...,[AS6] LBC424,"[658, 1747, 191, 166]",658,1747,191,166,AS,ASC-US,0,NaN,ASC-US
4,4,patch_images/2021.01.14/LBC424-20210111(1)/LBC...,[AS6] LBC424,"[1571, 365, 136, 146]",1571,365,136,146,AS,ASC-US,0,NaN,ASC-US


In [5]:
train_df.label_id.value_counts()

ASC-US          8162
LSIL            3821
Negative        1773
HSIL            1129
ASC-H            989
Candida          484
Endocervical     209
Carcinoma          1
Endometrial        1
Name: label_id, dtype: int64

In [6]:
len(train_df.label_id.unique())

9

In [7]:
class PapsClsModel(LightningModule) :
    def __init__(
        self,
        data_path : str,
        arch: str = 'resnet18',
        pretrained: bool = False,
        lr: float = 0.9,
        momentum: float = 0.9,
        weight_decay: float = 1e-4,
        batch_size: int =256,
        workers: int = 16,
        num_classes: int = 5,
    ):
        
        super().__init__()
        self.arch = arch
        self.pretrained = pretrained
        self.lr = lr
        self.momentum = momentum
        self.weight_decay = weight_decay
        self.data_path = data_path
        self.batch_size = batch_size
        self.workers = workers
        self.num_classes = num_classes
        
        # if args.arch not in models.__dict__.keys() : 
            # self.models = EfficientNet.from_name(args.arch)  
        self.models = custom_models.__dict__[self.arch](pretrained=False, img_size=448)
        # else :
        #     print('only resnet is supported') 
        #     self.models = models.__dict__[self.arch](pretrained=self.pretrained) 
        
        shape = self.models.fc.weight.shape
        self.models.fc = nn.Linear(shape[1], self.num_classes)
            
        # print("=> creating model '{}'".format(args.arch))
        self.train_dataset: Optional[Dataset] = None
        self.eval_dataset: Optional[Dataset] = None
        self.train_acc1 = Accuracy(top_k=1)
        self.eval_acc1 = Accuracy(top_k=1)
        
    def forward(self, x) :
        return self.models(x)
    
    def training_step(self, batch, batch_idx) :
        images, targets = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, targets)
        self.log('train_loss', loss)
        
        #update metric
        self.train_acc1(outputs, targets)
        self.log('train_acc', self.train_acc1, prog_bar=True)
        
        return loss
    
    def eval_step(self, batch, batch_idx, prefix: str) :
        images, targets = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, targets)
        self.log(f'{prefix}_loss', loss)
        self.eval_acc1(outputs, targets)
        self.log(f'{prefix}_acc1', self.eval_acc1, prog_bar=True)
        
    def validation_step(self, batch, batch_idx) :
        return self.eval_step(batch, batch_idx, 'val')
    
    def test_setup(self, batch, batch_idx) :
        return self.eval_step(batch, batch_idx, 'test')
    
    def configure_optimizers(self) :
        optimizer = optim.SGD(self.parameters(), lr=self.lr, momentum=self.momentum, weight_decay=self.weight_decay)
        scheduler = lr_scheduler.LambdaLR(optimizer, lambda epoch : 0.1 **(epoch //30))
        return [optimizer], [scheduler]

In [8]:
BATCH_SIZE=32
class PapsDataModule(LightningDataModule):
    def __init__(self, data_dir: str = '../lbp_data/'):
        super().__init__()
        self.data_dir = data_dir
        self.train_transform = train_transforms
        self.test_transform = test_transforms

        # self.dims is returned when you call dm.size()
        # Setting default dims here because we know them.
        # Could optionally be assigned dynamically in dm.setup()
        self.dims = (1, 28, 28)
        self.num_classes = 5

    def prepare_data(self):
        # download
        pass

    def setup(self, stage=None):

        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            train_df = pd.read_csv(self.data_dir + 'train.csv')
            self.train_dataset = PapsDataset(train_df, defaultpath=self.data_dir, transform=self.train_transforms)
            test_df = pd.read_csv(self.data_dir + 'test.csv')
            self.test_dataset = PapsDataset(test_df, defaultpath=self.data_dir, transform=self.test_transforms)            

        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            test_df = pd.read_csv(self.data_dir + 'test.csv')
            self.test_dataset = PapsDataset(test_df, defaultpath=self.data_dir, transform=self.test_transforms)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=BATCH_SIZE)

    def val_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=BATCH_SIZE)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=BATCH_SIZE)

In [9]:
dm = PapsDataModule()

/opt/conda/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:149: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


In [10]:
# images, labels = next(iter(dm.train_dataloader()))
# images.shape

In [11]:
img_size = 448

trainer_defaults = dict(
    # callbacks = [
    #     # the PyTorch example refreshes every 10 batches
    #     # TQDMProgressBar(refresh_rate=10),
    #     # save when the validation top1 accuracy improves
    #     ModelCheckpoint(monitor="val_acc1", mode="max"),
    # ],    
    # plugins = "deepspeed_stage_2_offload",
    precision = 16,
    max_epochs = 90,
    accelerator = 'gpu', # auto, or select device, "gpu"
    devices = 1, # number of gpus
    logger = True,
    benchmark = True,
    # strategy = "ddp",
    )

model = PapsClsModel(
    data_path='../lbp_data',
    arch='swin_t',
    pretrained=False,
    workers=8,
    lr = 0.01,
    batch_size=32,
    weight_decay=1e-4,
    num_classes=5)

trainer = Trainer(**trainer_defaults)
trainer.fit(model, dm)  

/opt/conda/lib/python3.9/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Using 16bit native Automatic Mixed Precision (AMP)


MisconfigurationException: `Trainer(strategy='ddp_spawn')` or `Trainer(accelerator='ddp_spawn')` is not compatible with an interactive environment. Run your code as a script, or choose one of the compatible strategies: Trainer(strategy=None|dp|tpu_spawn). In case you are spawning processes yourself, make sure to include the Trainer creation inside the worker function.